In [1]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)
import functions as fn
import random
from tqdm import tqdm
import pandas as pd
from nomic import embed
import time

/home/phaxssi/miniconda3/envs/rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/phaxssi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
from gpt4all import GPT4All
model = GPT4All("gpt4all-13b-snoozy-q4_0.gguf")

In [10]:
question = "What is the capital of France?"
context = "France is a country located in Europe. Its capital is known for its rich history and is a major global center for art, fashion, and culture."

In [11]:
prompt = f"""
You are an AI assistant specializing in Question Answering. Your task is to read the provided context carefully and then generate the most accurate and concise answer to the question based on the context. Provide only the answer, without repeating the question or adding unnecessary information. If the context does not contain the answer, respond with 'The answer is not available in the provided context.'

Context: {context}

Question: {question}

Answer:
"""

In [9]:
from gpt4all import GPT4All
model = GPT4All("gpt4all-13b-snoozy-q4_0.gguf")

In [52]:
url = "https://raw.githubusercontent.com/chen700564/RGB/master/data/en.json"
data = fn.process_json(url)
data = random.sample(data, 1)
queries = [item["query"] for item in data]
answers = [item["answer"][0] for item in data]

In [41]:
def get_number_tokens(text):
    return embed.text([text], inference_mode="local")['usage']['prompt_tokens']

In [54]:
for context in tqdm([item["positive"] for item in data], total=len(queries)):
    context_concat = "".join(context)
    content =  embed.text([context_concat], inference_mode="local", n_ctx=4000)

100%|██████████| 1/1 [00:05<00:00,  5.26s/it]


In [55]:
content

{'embeddings': [[0.06023503467440605,
   0.06160537526011467,
   -0.20606037974357605,
   0.016264650970697403,
   0.04228450730443001,
   0.004494164139032364,
   -0.012961332686245441,
   -0.005363604053854942,
   0.017922181636095047,
   -0.05528199300169945,
   -0.04742078483104706,
   -0.00605354318395257,
   0.04173861816525459,
   -0.04744865000247955,
   0.028957122936844826,
   -0.04121151939034462,
   0.018824191763997078,
   -0.03662305697798729,
   -0.06749533861875534,
   -0.03992370516061783,
   -0.05330648645758629,
   -0.02592475712299347,
   -0.012790714390575886,
   -0.003778270445764065,
   0.0981205403804779,
   -0.0241741631180048,
   -0.03695254772901535,
   0.007521060295403004,
   -0.03313879296183586,
   0.00833171047270298,
   0.1013156995177269,
   -0.03210475668311119,
   -0.061445239931344986,
   -0.04729089140892029,
   -0.03754235804080963,
   -0.07154868543148041,
   0.03994548320770264,
   0.06765153259038925,
   -0.012425950728356838,
   -0.00735894544

In [39]:
for context in tqdm([item["positive"] for item in data], total=len(queries)):
    context_concat = "".join(context)
    test = embed.text([context_concat], inference_mode="local")

100%|██████████| 1/1 [00:05<00:00,  5.21s/it]


In [37]:
for query, context, answer in tqdm(zip(queries, [item["positive"] for item in data], answers), total=len(queries)):
    context_concat = "".join(context)
    with model.chat_session():
        prompt = f"""
        You are an AI assistant specializing in Question Answering. Your task is to read the provided context carefully and then generate the most accurate and concise answer to the question based on the context.'

        Context: {context_concat}

        Question: {query}

        Answer:
        """
        number_tokens = get_number_tokens(prompt)
        print(f"Number of tokens: {number_tokens}")

100%|██████████| 1/1 [00:05<00:00,  5.01s/it]

Number of tokens: 2042


In [35]:
prompt

'\n        You are an AI assistant specializing in Question Answering. Your task is to read the provided context carefully and then generate the most accurate and concise answer to the question based on the context.\'\n\n        Context: Tyler Adams will captain the US men’s national team into the 2022 FIFA World Cup, head coach Gregg Berhalter announced on the eve of Monday’s Group B opener vs. Wales (2 pm ET | FOX, Telemundo). The Leeds United midfielder and New York Red Bulls homegrown product will be the USMNT’s youngest captain since Walter Bahr at the 1950 World Cup. He’s already worn the armband nine times during 32 career international appearances. “We think he has great leadership capabilities,” Berhalter said of the 23-year-old Wappingers Falls, New York native. “He leads by his actions and his words.” FIFA requires teams to name a singular captain for the tournament; the USMNT were the last of 32 Qatar-qualified squads to complete that task. During Berhalter’s tenure, the US

In [47]:
results = []
for query, context, answer in tqdm(zip(queries, [item["positive"] for item in data], answers), total=len(queries)):
    context_concat = "".join(context)

    result = {
        'Query': query,
        'Correct Answer': answer,
    }
    with model.chat_session():
        prompt = f"""
        You are an AI assistant specializing in Question Answering. Your task is to read the provided context carefully and then generate the most accurate and concise answer to the question based on the context.'

        Context: {context_concat}

        Question: {query}

        Answer:
        """
        number_tokens = get_number_tokens(prompt)
        print(f"Number of tokens: {number_tokens}")
        generated_answer = model.generate(prompt)
        result.update({'Generated Answer': generated_answer})   
        results.append(result)

  0%|          | 0/1 [00:00<?, ?it/s]

Number of tokens: 692


100%|██████████| 1/1 [02:38<00:00, 158.95s/it]


In [17]:
results = []
enc = tiktoken.encoding_for_model("gpt-3.5-turbo") 
max_tokens = 2048 
for query, context, answer in tqdm(zip(queries, [item["positive"] for item in data], answers), total=len(queries)):
    context_concat = "".join(context)
    prompt_prefix = f"""
    You are an AI assistant specializing in Question Answering. Your task is to read the provided context carefully and then generate the most accurate and concise answer to the question based on the context.'

    Context:
    """

    prompt_tokens_lenght = len(enc.encode(context_concat))

    prompt_tokens = enc.encode(context_concat + prompt_prefix + query + "\n\nAnswer:")
    prompt_token_length = len(prompt_tokens)
    print(prompt_token_length)


100%|██████████| 1/1 [00:00<00:00, 285.27it/s]

3669


In [58]:
import tiktoken
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [57]:
prompt_template = """
You are an AI assistant specializing in Question Answering. Your task is to read the provided context carefully and then generate the most accurate and concise answer to the question based on the context.

Context: {context_concat}

Question: {query}

Answer:
"""

In [59]:
prompt_tokens = enc.encode(prompt_template)
prompt_token_length = len(prompt_tokens)
print(f'Prompt token length: {prompt_token_length}')

Prompt token length: 51


In [68]:
results = []
max_total_tokens = 1650

results = []
for query, context, answer in tqdm(zip(queries, [item["positive"] for item in data], answers), total=len(queries)):
    context_concat = "".join(context)
    context_tokens = enc.encode(context_concat)
    limited_tokens = context_tokens[:max_total_tokens]
    context_concat_limited = enc.decode(limited_tokens)

    result = {
        'Query': query,
        'Correct Answer': answer,
    }
    with model.chat_session():
        prompt = f"""
        You are an AI assistant specializing in Question Answering. Your task is to read the provided context carefully and then generate the most accurate and concise answer to the question based on the context.'

        Context: {context_concat_limited}

        Question: {query}

        Answer:
        """
        prompt_tokens_length = len(enc.encode(prompt))
        print(f"Prompt tokens length: {prompt_tokens_length}")

        generated_answer = model.generate(prompt)
        result.update({'Generated Answer': generated_answer})   
        results.append(result)

  0%|          | 0/1 [00:00<?, ?it/s]

Prompt tokens length: 1707


LLaMA: reached the end of the context window so resizing
100%|██████████| 1/1 [06:43<00:00, 403.94s/it]


In [64]:
results

[{'Query': 'Where did Leonard Francis escape from?',
  'Correct Answer': ['San Diego'],
  'Generated Answer': 'ERROR: The prompt size exceeds the context window size and cannot be processed.\n #include <iostream>\n#include the following libraries:\n```c++\n#include <string>\ninclude <vector>\ninclude <algorithm>\ninclude <unordered_map>\n```'}]

In [39]:
results

[{'Query': 'When will the final season of The Blacklist premiere?',
  'Correct Answer': ['February 26',
   'Feb 26',
   'Feb. 26',
   '26 February',
   '26 Feb',
   '26 Feb.'],
  'Generated Answer': 'and00000000000000002000000000000000000'}]

In [18]:
prompt_positive = """
You are an AI assistant specializing in Question Answering. Your task is to read the provided context carefully and then generate the most accurate and concise answer to the question based on the context.'

Context: {positive_context_concat}

Question: {query}

Answer:
"""


prompt_negative = """
You are an AI assistant specializing in Question Answering. Your task is to read the provided context carefully and then generate the most accurate and concise answer to the question based on the context.'

Context: {negative_context_concat}

Question: {query}

Answer:
"""


prompt_posneg= """
You are an AI assistant specializing in Question Answering. Your task is to read the provided context carefully and then generate the most accurate and concise answer to the question based on the context.'

Context: {posneg_context_concat}

Question: {query}

Answer:
"""

In [ ]:
gen_model_1 = GPT4All("gpt4all-13b-snoozy-q4_0.gguf", device="cuda")
gen_model_2 = GPT4All("gpt4all-13b-snoozy-q4_1.gguf", device="cuda")

models = [gen_model_1, gen_model_2]

separator = " <|> "

context_types = {
    'Positive': lambda data: separator.join([item["positive"] for item in data]),
    'Negative': lambda data: separator.join([item["negative"] for item in data]),
    'PosNeg': lambda data: separator.join([item["positive"] for item in data]) + separator + separator.join([item["negative"] for item in data])
}

prompts = {
    'Positive': prompt_positive,
    'Negative': prompt_negative,
    'PosNeg': prompt_posneg
}

for model in models:
    
    results = []
    for query, data, answer in tqdm(zip(queries, data_list, answers), total=len(queries)):
        
        result = {
            'Query': query,
            'Correct Answer': answer,
        }
        
        with model.chat_session():
            for context_type, context_func in context_types.items():
                context_concat = context_func([data])
                prompt = prompts[context_type].format(context_concat=context_concat, query=query)
                generated_answer = model.generate(prompt)
                result[f'{context_type} Predicted Answer'] = generated_answer   
            
            results.append(result)

    results_df = pd.DataFrame(results)
    results_df['EM Positive'] = results_df.apply(lambda row: fn.apply_exact_match(row, 'Positive Predicted Answer', 'Correct Answer'), axis=1)
    results_df['EM Negative'] = results_df.apply(lambda row: fn.apply_exact_match(row, 'Negative Predicted Answer', 'Correct Answer'), axis=1)
    results_df['EM PosNeg'] = results_df.apply(lambda row: fn.apply_exact_match(row, 'PosNeg Predicted Answer', 'Correct Answer'), axis=1)

    results_df['Jaccard Positive'] = results_df.apply(lambda row: fn.apply_jaccard(row, 'Positive Predicted Answer', 'Correct Answer'), axis=1)
    results_df['Jaccard Negative'] = results_df.apply(lambda row: fn.apply_jaccard(row, 'Negative Predicted Answer', 'Correct Answer'), axis=1)
    results_df['Jaccard PosNeg'] = results_df.apply(lambda row: fn.apply_jaccard(row, 'PosNeg Predicted Answer', 'Correct Answer'), axis=1)

    results_df['Cosine Positive'] = results_df.apply(lambda row: fn.apply_cosine(row, 'Positive Predicted Answer', 'Correct Answer'), axis=1)
    results_df['Cosine Negative'] = results_df.apply(lambda row: fn.apply_cosine(row, 'Negative Predicted Answer', 'Correct Answer'), axis=1)
    results_df['Cosine PosNeg'] = results_df.apply(lambda row: fn.apply_cosine(row, 'PosNeg Predicted Answer', 'Correct Answer'), axis=1)

    # Save results
    filename = f"/content/drive/My Drive/tornado-tasks/results/{model.split('/')[1]}.json"
    results_df.to_json(filename, orient='records', lines=True)

end_time = time.time()
print(f"Total execution time: {(end_time - start_time) / 60} minutes.")

In [ ]:
gen_model_1 = GPT4All("gpt4all-13b-snoozy-q4_0.gguf", device="cuda")
gen_model_2 = GPT4All("gpt4all-13b-snoozy-q4_1.gguf", device="cuda")

models = [gen_model_1, gen_model_2]

separator = " <|> "

for model in models:
    
    results = []
    for query, positive_context, negative_context, answer in tqdm(zip(queries, [item["positive"] for item in data], [item["negative"] for item in data], answers), total=len(queries)):
        positive_context_concat = separator.join(positive_context)
        negative_context_concat = separator.join(negative_context)
        posneg_context_concat = positive_context_concat + separator + negative_context_concat

        result = {
            'Query': query,
            'Correct Answer': answer,
        }
        with model.chat_session():
            prompt = prompt_positive.format(positive_context_concat=positive_context_concat, query=query)
            generated_answer = model.generate(prompt)
            result.update({'Positive Predicted Answer': generated_answer})   
            results.append(result)

    results_df = pd.DataFrame(results)

In [ ]:
model = GPT4All("gpt4all-13b-snoozy-q4_0.gguf", device="cuda")
results = []
for query, context, answer in tqdm(zip(queries, [item["positive"] for item in data], answers), total=len(queries)):
    context_concat = "".join(context)
    context_concat = context_concat[:5000]

    result = {
        'Query': query,
        'Correct Answer': answer,
    }
    with model.chat_session():
        prompt = f"""
        You are an AI assistant specializing in Question Answering. Your task is to read the provided context carefully and then generate the most accurate and concise answer to the question based on the context.'

        Context: {context_concat}

        Question: {query}

        Answer:
        """
        generated_answer = model.generate(prompt)
        result.update({'Generated Answer': generated_answer})   
        results.append(result)
